## Build an Agent <img src="../../images/db-icon.png" width=25 />

By themselves, language models can't take actions - they just output text. A big use case for LangChain is creating agents. Agents are systems that use LLMs as reasoning engines to determine which actions to take and the inputs necessary to perform the action. After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish. This is often achieved via tool-calling.

In this tutorial we will build an agent that can interact with a search engine. You will be able to ask this agent questions, watch it call the search tool, and have conversations with it.

### LangSmith

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls. As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent. The best way to do this is with LangSmith.

After you sign up at the link above, make sure to set environment variables to start logging traces:

```LANGCHAIN_TRACING_V2=true```<br>
```LANGCHAIN_API_KEY="..."```<br>
```LANGSMITH_PROJECT=tutorial```

In [ ]:
from dotenv import load_dotenv

# load environmental variables
load_dotenv()

### Tools

#### Tavily
We will be using [Tavily](https://app.tavily.com/home) (a search engine) as a tool. In order to use it, you will need to get and set an API key.

#### Define tools
We first need to create the tools we want to use. Our main tool of choice will be Tavily - a search engine. We have a built-in tool in LangChain to easily use Tavily search engine as tool.

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

#### Using LLMs with Tools
Next, let's learn how to use a language model to call tools. LangChain supports many different language models that you can use interchangably

In [ ]:
from databricks_langchain import ChatDatabricks

model = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct", temperature=0)

We can now see what it is like to enable this model to do tool calling. In order to enable that we use .bind_tools to give the language model knowledge of these tools

In [ ]:
model_with_tools = model.bind_tools(tools)

We can now call the model. Let's first call it with a normal message, and see how it responds. We can look at both the ```content``` field as well as the ```tool_calls``` field.

In [ ]:
from langchain_core.messages import HumanMessage

response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

Now, let's try calling it with some input that would expect a tool to be called.

In [ ]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

We can see that there's now no text content, but there is a tool call! It wants us to call the Tavily Search tool.

This isn't calling that tool yet - it's just telling us to. In order to actually call it, we'll want to create our agent.

### Agents

#### Create Agent
Now that we have defined the tools and the LLM, we can create the agent. We will be using LangGraph to construct the agent. Currently, we are using a high level interface to construct the agent, but the nice thing about LangGraph is that this high-level interface is backed by a low-level, highly controllable API in case you want to modify the agent logic.

Now, we can initialize the agent with the LLM and the tools.

Note that we are passing in the ```model```, not ```model_with_tools```. That is because ```create_react_agent``` will call ```.bind_tools``` for us under the hood.

In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(model, tools)

In [ ]:
from IPython.display import Image, display

try:
    display(Image(agent.get_graph().draw_mermaid_png()))
except Exception:
    pass

#### Run Agent
We can now run the agent with a few queries! Note that for now, these are all stateless queries (it won't remember previous interactions). Note that the agent will return the final state at the end of the interaction (which includes any inputs, we will see later on how to get only the outputs).

First up, let's see how it responds when there's no need to call a tool:

In [ ]:
response = agent.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

In order to see exactly what is happening under the hood (and to make sure it's not calling a tool) we can take a look at the [LangSmith trace](https://smith.langchain.com/public/28311faa-e135-4d6a-ab6b-caecf6482aaa/r).

Let's now try it out on an example where it should be invoking the tool

In [ ]:
response = agent.invoke({"messages": [HumanMessage(content="whats the weather in sf?")]})

response["messages"]

We can check out the [LangSmith trace](https://smith.langchain.com/public/28311faa-e135-4d6a-ab6b-caecf6482aaa/r) to make sure it's calling the search tool effectively.

#### Streaming Messages
We've seen how the agent can be called with .invoke to get a final response. If the agent executes multiple steps, this may take a while. To show intermediate progress, we can stream back messages as they occur.

In [ ]:
for step in agent.stream({"messages": [HumanMessage(content="whats the weather in sf?")]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

#### Streaming Tokens
In addition to streaming back messages, it is also useful to stream back tokens. We can do this by specifying stream_mode="messages".

In [ ]:
for step, metadata in agent.stream({"messages": [HumanMessage(content="whats the weather in sf?")]}, stream_mode="messages"):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

#### Adding Memory
As mentioned earlier, this agent is stateless. This means it does not remember previous interactions. To give it memory we need to pass in a checkpointer. When passing in a checkpointer, we also have to pass in a thread_id when invoking the agent (so it knows which thread/conversation to resume from)

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [ ]:
agent = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
for chunk in agent.stream({"messages": [HumanMessage(content="What is the weather in sf?")]}, config):
    print(chunk)
    print("----")

In [ ]:
for chunk in agent.stream({"messages": [HumanMessage(content="What should I wear?")]}, config):
    print(chunk)
    print("----")

### Conclusion
That's a wrap! In this quick start we covered how to create a simple agent. We've then shown how to stream back a response - not only with the intermediate steps, but also tokens! We've also added in memory so you can have a conversation with them. Agents are a complex topic with lots to learn!